# Introduction

**Incarceration & COVID-19: How Jails Respond to COVID**<br>

This project scrapes daily jail roster information to create a large dataset. This dataset is designed to analyze how jail populations have fluctuated in response to COVID-19. Research centers on explaining why county jails in different parts of the United States have responded differently to the pandemic over time. 

A separate but related idea for this dataset analyzes the impact of pandemic-related jail population declines on local crime. This project uses daily jail roster population counts as the focal variable and the analysis uses group-based trajectory modeling. Our scraped data will address gaps in the [Vera](https://github.com/vera-institute/jail-population-data) dataset.

We start by comparing Washington and New York states because they dealt with COVID-19 at the early on-set of the pandemic. Below is a list of the data points to collect to harmonize with the Vera data.
- County Name
- State Name
- Daily Population Counts
- Reporting Jail Name

# Imports

In [2]:
# Import libraries
import pandas as pd
from datetime import datetime

# Request
import requests
import re
import zlib
import urllib
import urllib.request

# Selenium
from selenium import webdriver

#SQL
import mysql.connector
import os

# MySQL

In [12]:
db_user = os.getenv('db_user')
db_passwd = os.getenv('db_passwd')


mydb = mysql.connector.connect(host='localhost',\
                              user=db_user,\
                              passwd=db_passwd,\
                              database='testdb'\
                              )
                               
print(mydb)

In [14]:
# Create a database
mycursor = mydb.cursor()

# This line is commented out because it only needs to be run once
# mycursor.execute("CREATE DATABASE testdb")

mycursor.execute("SHOW DATABASES")

for db in mycursor:
    print(db)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('testdb',)


In [16]:
# # Create a table (do not erase/for refrerence)

# mycursor.execute("CREATE TABLE county_jails\
#                  (reporting_jurisdictions VARCHAR(100),\
#                  county_name VARCHAR(100),\
#                  state_name VARCHAR(100),\
#                  Date VARCHAR(100),\
#                  jail_population INTEGER(255))")

mycursor.execute("SHOW TABLES")

for tb in mycursor:
    print(tb)

('county_jails',)


In [17]:
sqlFormula = "INSERT INTO county_jails (Date, reporting_jurisdictions, county_name, \
state_name, jail_population) VALUES (%s, %s, %s, %s, %s)"

# States

Be sure to check for APIs in addition to scraping. Will include NY, WA and FL.

## Washington

### Whitman

In [ ]:
#View website with driver
url = "http://www.whitmancountyjail.org/"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

#Scrape the county jail daily count, view full list 
listy = driver.find_elements_by_css_selector('h4')
# for x in listy[:50]:
#     if len(x.text) > 0:
#         print(x.text)

#Identify and create dynamic fields
todays_date = datetime.now().strftime('%Y-%m-%d')
JPWhitman = (len(listy))-10
print('Date = ',todays_date)
print('jail_population = ',JPWhitman)


#Use for all MySQL pushes
Whitman = (todays_date, "Whitman County Jail", "Whitman County", "WA", JPWhitman)
mycursor.execute(sqlFormula, Whitman)
mydb.commit()

#Exit out of browser window
driver.close()

### Spokane

In [ ]:
url = "https://www.spokanecounty.org/352/Inmate-Roster'"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

# Sinmate = driver.find_element_by_xpath('//*[@id="tblInmateRoster_info"]')

In [ ]:
JPSpokane = 626

In [ ]:
#USE FOR ALL COMMITS
Spokane = (todays_date, "Spokane County Jail", "Spokane County", "WA", JPSpokane)
mycursor.execute(sqlFormula, Spokane)
mydb.commit()

driver.close()

### Okanogan

Details can be found in the Daily Jail Inmate Log on [Okanogan Sherriff Website](https://okanogansheriff.org/).

In [ ]:
url = "https://okanogansheriff.org/"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [ ]:
JPOkanogan = 88

In [ ]:
#USE FOR ALL COMMITS
Okanogan = (todays_date, "Okanogan County Jail", "Okanogan County", "WA", JPOkanogan)
mycursor.execute(sqlFormula, Okanogan)
mydb.commit()

driver.close()

### Jefferson

[Jefferson](https://co.jefferson.wa.us/174/Jail-Inmate-Search)<br> To view the full inmate roster click the Clear button then the Search button.

In [ ]:
url = "https://co.jefferson.wa.us/174/Jail-Inmate-Search"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

In [ ]:

# Hidden input type

inmate = driver.find_elements_by_name('Name')
print(len(inmate))

In [ ]:
JPJefferson = 22

In [ ]:
#USE FOR ALL COMMITS
Jefferson = (todays_date, "Jefferson County Jail", "Jefferson County", "WA", JPJefferson)
mycursor.execute(sqlFormula, Jefferson)
mydb.commit()

driver.close()

### Grant

[Grant](ttps://www.grantcountywa.gov/SHERIFF/Corrections/Inmate-Roster.htm), daily pdf

In [ ]:
grant = urllib.request.Request("https://www.grantcountywa.gov/SHERIFF/Corrections/Roster-InmateinmateRoster%20v%206.rpt.pdf",\
#                               method= 'GET',\
                               headers= { 'User-Agent' : 'Chrome/41.0.2228.0',\
                                 'Content-Type': 'application/x-www-form-urlencoded'\
                                        })

response = urllib.request.urlopen(grant)
the_page = response.read()
# print(the_page)

In [ ]:
stream = re.compile(b'.*?FlateDecode.*?stream(.*?)endstream', re.S)

for s in re.findall(stream,the_page):
    s = s.strip(b'\r\n')
    try:
        print(zlib.decompress(s).decode('UTF-8'))
    except:
        pass
    

In [ ]:
JPGrant=43

In [ ]:
#USE FOR ALL COMMITS
Grant = (todays_date, "Grant County Jail", "Grant County", "WA", JPGrant)
mycursor.execute(sqlFormula, Grant)
mydb.commit()

# driver.close()

### Gray's Harbor

[Gray's Harbor]('http://ghlea.com/JailRosters/GHCJRoster.html')

In [ ]:
url = "http://ghlea.com/JailRosters/GHCJRoster.html"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)


GHinmate = driver.find_elements_by_xpath('//*[@id="main-table"]/tbody/tr')
JPGray=(len(GHinmate))
print(JPGray)

#USE FOR ALL COMMITS

Gray = (todays_date, "Grays Harbor County Jail", "Grays Harbor County", "WA", JPGray)
mycursor.execute(sqlFormula, Gray)
mydb.commit()


driver.close()

### Ferry

[Ferry](https://www.ferry-county.com/Courts%20and%20Law/Inmate%20Roster/Inmate_Roster_Page.html): in the section that says "MAY 11, 2020 - 8 inmates")

In [ ]:
url = "https://www.ferry-county.com/Courts%20and%20Law/Inmate%20Roster/Inmate_Roster_Page.html"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)


Finmate = driver.find_element_by_xpath('//*[@id="mainContent3"]/p[9]').text
JPFerry=Finmate[15:17]
print(JPFerry)


#USE FOR ALL COMMITS

Ferry = (todays_date, "Ferry County Corrections", "Ferry County", "WA", JPFerry)
mycursor.execute(sqlFormula, Ferry)
mydb.commit()


driver.close()

### Clallam

[Clallam](https://websrv23.clallam.net/NewWorld.InmateInquiry/WA0050000/)

In [ ]:
url = "https://websrv23.clallam.net/NewWorld.InmateInquiry/WA0050000/"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver') 
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)


Clallam_inmate = driver.find_elements_by_class_name('Name')
JPClallam = (len(Clallam_inmate))
print(JPClallam)


#USE FOR ALL COMMITS

Clallam = (todays_date, "Clallam County Jail", "Clallam County", "WA", JPClallam)
mycursor.execute(sqlFormula, Clallam)
mydb.commit()


driver.close()

### Adams

[View](https://www.co.adams.wa.us/government/jail_roster_and_booking_information/index.php) Jail Roster Information

In [ ]:
url = "https://www.co.adams.wa.us/jailrosterout.txt"
driver = webdriver.Chrome('/Users/meaganrossi/Projects/Incarceration_COVID/chromedriver')
driver.implicitly_wait(3)
driver.get(url)

# print(driver.page_source)

Adams_text=driver.find_element_by_xpath('/html/body/pre').text
JPAdams = Adams_text.count("Booking")
print('Jail Population = ',JPAdams)

#USE FOR ALL COMMITS

Adams = (todays_date, "Adams County Jail", "Adams County", "WA", JPAdams)
mycursor.execute(sqlFormula, Adams)
mydb.commit()

driver.close()

### Chelan

In [ ]:
#  https://www.co.chelan.wa.us/regional-jail/inmate-list

### Cowlitz

In [ ]:
# Cowlitz http://apps.co.cowlitz.wa.us/CCCD/Custody/default/Index.html

### Franklin

In [ ]:
# Franklin http://apps.co.cowlitz.wa.us/CCCD/Custody/default/Index.html
# (pdf, no anumber that i saw, would need to be counted somehow)

### Kitsap

In [ ]:
# Kitsap https://www.kitsapgov.com/sheriff/Pages/InCustody.aspx

### Kittitas

In [ ]:
#  https://www.co.kittitas.wa.us/sheriff/roster.aspx
# (irregular, hard to count?)

### Lewis

In [ ]:
#  https://jail.lewiscountywa.gov/

### Mason

In [ ]:
#  https://so.co.mason.wa.us/documents/incustdy.pdf
# pdf, hard?

### Pierce

In [ ]:
#  https://linxonline.co.pierce.wa.us/linxweb/Booking/GetJailRoster.cfm
# I had to go to it once and get a log in page, then go back, then return, and i could see the roster without logging in

### Skagit

In [ ]:
#  https://www.skagitcounty.net/Reporting/JailRoster/
# has the total #, easier?

### Skamania

In [ ]:
#  (amazing name) http://skamaniasheriff.com/corrections/daily-population/

### Whatcom

In [ ]:
#  https://apps1.whatcomcounty.us/jaildata/roster.html
# list, no summary #

### Yakima

In [ ]:
#  http://www.yakimaco.us/inmatelookup/YcDocPublicIncarcerated.aspx
# hard one

### Benton

In [ ]:
#  http://apps.co.cowlitz.wa.us/CCCD/Custody/default/Index.html

### Klickitat

In [ ]:
#  https://www.klickitatcounty.org/DocumentCenter/View/1416/Booking-Roster-PDF
# pdf

### Stevens

In [ ]:
#  https://sheriff.stevenscountywa.gov/jail/inmate-roster/
# list of pdfs

### Wahkiakum

In [ ]:
#  http://jailviewer.co.wahkiakum.wa.us/Home/BookingSearchQuery?
# that's a search by name, not roster of all, but adding it just in case you can see the population on the back end!

### Pacific

In [ ]:
#  https://co.pacific.wa.us/sheriff/corrections/

### Thurston

In [ ]:
#  https://www.co.thurston.wa.us/sheriff/bureau-corrections-roster-search.asp?mod=fourth

In [ ]:
# WA Counties with No Website List
# Douglas
# Garfield
# Pend Oreille
# San Juan
# Asotin

# Export csv

In [ ]:
country_jail_df = pd.read_sql("SELECT DISTINCT * FROM county_jails", con=mydb)
country_jail_df.head()

In [ ]:
country_jail_df.to_csv('County_Jail.csv')

In [ ]:
country_jail_df.shape